<a href="https://colab.research.google.com/github/LisethTiria/Biosenales-2025-1/blob/main/practica_3_parte_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
# Importa las bibliotecas necesarias
import numpy as np  # Para cálculos numéricos con matrices
import pandas as pd  # Para el manejo y manipulación de datos con DataFrames
import matplotlib.pyplot as plt  # Para crear visualizaciones
import scipy.io as sio  # Para leer y escribir archivos de datos (por ejemplo, archivos .mat)
import scipy.signal  # Para tareas de procesamiento de señales
from scipy.signal import find_peaks  # Para la detección de picos en señales
from scipy import signal # Para procesamiento de señales
from scipy.stats import levene, ttest_ind, mannwhitneyu, shapiro  # Para pruebas estadísticas
from google.colab import drive  # Para montar Google Drive en Colab
import os  # Para interactuar con el sistema operativo (por ejemplo, rutas de archivos)
from glob import glob  # Para encontrar archivos que coincidan con un patrón


In [15]:
# Monta Google Drive para acceder a los archivos de datos
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
# Función para calcular la energía promedio por canal
def energia_promedio_por_canal(signal):
    """
    Calcula la energía promedio por canal de una señal multicanal y multiépoca.

    Entrada:
        - signal: ndarray de forma (n_canales, n_muestras, n_épocas)

    Salida:
        - Energías promedio por canal (matriz de tamaño n_canales)
    """
    energia_por_epoca = np.sum(signal**2, axis=1)  # Energía por época, por canal
    energia_promedio = np.mean(energia_por_epoca, axis=1)  # Promedio entre épocas
    return energia_promedio

In [17]:
# Procesa los datos para el grupo de control
carpeta = "/content/drive/MyDrive/datos_senales_datos_parkinson_cursos/control"  # Ruta a los datos del grupo de control
archivos = glob(os.path.join(carpeta, "*.mat"))  # Obtiene todos los archivos .mat en la carpeta

print(f"{len(archivos)} archivos encontrados.")  # Imprime el número de archivos encontrados

def calcular_energia_promedio_por_canal(data):
    """Calcula la energía promedio por canal."""
    energia = np.mean(np.sum(data**2, axis=1), axis=1)
    return energia

lista_energias = []  # Almacena vectores de energía para cada sujeto
nombres_sujetos = []  # Almacena los nombres de los sujetos

for archivo in archivos:  # Itera a través de cada archivo
    mat = scipy.io.loadmat(archivo)  # Carga datos del archivo .mat
    if 'data' in mat:
        data = mat['data']
        energia = calcular_energia_promedio_por_canal(data)
        lista_energias.append(energia)
        nombre = os.path.basename(archivo).split('.')[0]
        nombres_sujetos.append(nombre)
    else:
        print(f" No se encontró la variable 'data' en {archivo}")

# Crea un DataFrame para las energías del grupo de control
df_energias = pd.DataFrame(lista_energias, columns=[f'canal_{i+1}' for i in range(data.shape[0])])
df_energias.index = nombres_sujetos
df_energias.index.name = 'sujeto'
print(df_energias.head())  # Muestra las primeras filas del DataFrame
df_energias.to_csv("energia_sujetos_control.csv")  # Guarda en CSV

36 archivos encontrados.
                              canal_1       canal_2       canal_3  \
sujeto                                                              
C001R_EP_reposo          21465.650358  20985.907912  22760.149588   
C002_EP_reposo           15966.402868  17617.810248  20804.937129   
C004_EP_reposo           14148.673322  18283.999666  28749.932148   
C005_EP_reposo_Repetido  35311.301696  34916.686010  38800.429029   
C006_EP_reposo           18510.829979  19738.489375  20911.792748   

                              canal_4       canal_5        canal_6  \
sujeto                                                               
C001R_EP_reposo          18505.640284  29730.163026   25244.158073   
C002_EP_reposo           19654.400017  16678.982063   93894.049009   
C004_EP_reposo           14270.726911  28787.445978   14661.417740   
C005_EP_reposo_Repetido  35427.031127  35905.472869  106598.128152   
C006_EP_reposo           21828.254399  23351.992649   53086.059766   



In [18]:
# Procesa los datos para el grupo de Parkinson (similar al grupo de control)
carpeta2 = "/content/drive/MyDrive/datos_senales_datos_parkinson_cursos/parkinson"
archivos2 = glob(os.path.join(carpeta2, "*.mat"))

print(f"{len(archivos2)} archivos encontrados.")

def calcular_energia_promedio_por_canal(data):
    """Calcula la energía promedio por canal."""
    energia2 = np.mean(np.sum(data**2, axis=1), axis=1)
    return energia2

lista_energias2 = []
nombres_sujetos2 = []

for archivo2 in archivos2:
    mat = scipy.io.loadmat(archivo2)
    if 'data' in mat:
        data2 = mat['data']
        energia2 = calcular_energia_promedio_por_canal(data2)
        lista_energias2.append(energia2)
        nombre2 = os.path.basename(archivo2).split('.')[0]
        nombres_sujetos2.append(nombre2)
    else:
        print(f"⚠️ No se encontró la variable 'data' en {archivo2}")

# Crea un DataFrame para las energías del grupo de Parkinson
df_energias2 = pd.DataFrame(lista_energias2, columns=[f'canal_{i+1}' for i in range(data2.shape[0])])
df_energias2.index = nombres_sujetos2
df_energias2.index.name = 'sujeto'
print(df_energias2.head())
df_energias2.to_csv("energia_sujetos_parkinson.csv")

23 archivos encontrados.
                     canal_1       canal_2       canal_3       canal_4  \
sujeto                                                                   
P001_EP_reposo  12438.243570  11261.175800  10819.634775   9489.784462   
P004_EP_reposo  17995.660058  12001.601821  12286.344400  14785.908284   
P005_EP_reposo  38092.102574  43575.379457  41979.994799  41715.287990   
P007_EP_reposo  23742.325612  22070.007569  24540.315612  21803.936448   
P012_EP_reposo  48574.518921  51806.529769  73171.952374  59707.699631   

                     canal_5        canal_6        canal_7        canal_8  
sujeto                                                                     
P001_EP_reposo  12091.060945   22798.213463   23700.620349   25606.065340  
P004_EP_reposo  17058.433161   63983.449318   53715.460772   66403.639479  
P005_EP_reposo  46513.737045  251649.394709  179345.438488  262361.180410  
P007_EP_reposo  22594.339745  128314.264805  128888.485633  152799.284248  


In [19]:
# Realiza una comparación estadística entre los grupos


df_control = pd.read_csv('/content/energia_sujetos_control.csv')  # Carga los datos del grupo de control
df_parkinson = pd.read_csv('/content/energia_sujetos_parkinson.csv')  # Carga los datos del grupo de Parkinson

canales = df_control.columns.intersection(df_parkinson.columns).drop('sujeto', errors='ignore')  # Obtiene los canales comunes
resultados = []  # Almacena los resultados de la prueba estadística

for canal in canales:  # Itera a través de cada canal
    grupo1 = df_control[canal].dropna()
    grupo2 = df_parkinson[canal].dropna()

    # Prueba de normalidad (Shapiro-Wilk)
    p_norm1 = shapiro(grupo1).pvalue
    p_norm2 = shapiro(grupo2).pvalue
    normal1 = p_norm1 > 0.05
    normal2 = p_norm2 > 0.05

    # Prueba de homogeneidad de varianzas (Levene)
    p_levene = levene(grupo1, grupo2).pvalue
    iguales_varianzas = p_levene > 0.05

    # Elige la prueba estadística apropiada
    if normal1 and normal2 and iguales_varianzas:
        stat, p_valor = ttest_ind(grupo1, grupo2, equal_var=True)
        prueba = "t (varianzas iguales)"
    elif normal1 and normal2:
        stat, p_valor = ttest_ind(grupo1, grupo2, equal_var=False)
        prueba = "t de Welch"
    else:
        stat, p_valor = mannwhitneyu(grupo1, grupo2, alternative='two-sided')
        prueba = "mann-whitney"

    resultados.append({
        "canal": canal,
        "p_normal_control": p_norm1,
        "p_normal_parkinson": p_norm2,
        "p_levene": p_levene,
        "prueba_usada": prueba,
        "p_valor": p_valor,
        "diferencia_significativa": p_valor < 0.05
    })

# Crea un DataFrame de resultados
df_resultados = pd.DataFrame(resultados)

# Muestra los canales con diferencias significativas
print("\nCanales con diferencia significativa entre grupos:")
print(df_resultados[df_resultados["diferencia_significativa"]][["canal", "prueba_usada", "p_valor"]])

# Guarda los resultados en CSV
df_resultados.to_csv("comparacion_estadistica_entre_grupos1.csv", index=False)


Canales con diferencia significativa entre grupos:
Empty DataFrame
Columns: [canal, prueba_usada, p_valor]
Index: []
